# Building sub-pipelines

---

[$facet](https://docs.mongodb.com/manual/reference/operator/aggregation/facet/) - Processing multiple aggregation sub-pipelines within a single stage on the same set of input documents.

---
### Conneting to MongoDB using Pymongo
----

In [1]:
# Importing the required libraries
import pymongo
import pprint as pp

pp.sorted = lambda x, key=None: x

In [2]:
# Connect to the mongo client - Atlas Cluster
client = pymongo.MongoClient('mongodb://localhost:27017/')

In [3]:
# Database
db = client.training

In [4]:
# Sample document
pp.pprint(
    db.hr.find_one())

{'_id': ObjectId('60af5db0b2f5ad99212f9464'),
 'enrollee_id': 23798,
 'gender': 'Male',
 'date_of_enrollment': datetime.datetime(2016, 1, 23, 0, 0),
 'city': {'name': 'city_149', 'development_index': 0.689},
 'education': {'level': 'Graduate', 'discipline': 'STEM'},
 'experience': {'years': 3,
                'company_type': 'Pvt Ltd',
                'last_new_job': 1,
                'relevent_experience': 1},
 'training_hours': 106}


---
### Question - 

Find average training hours delivered per year and per month after 2017.

----

**Importing datetime library**

----

In [5]:
# Import library
from datetime import datetime

In [6]:
# Question
result = db.hr.aggregate(
                    [
                        # Stage 1 - Filter documents
                        {
                            '$match':{'date_of_enrollment':{'$gte':datetime(2018,1,1)}}
                        },
                        # Stage 2 - Project year
                        {
                            '$project':{
                                            'Year':{'$year':'$date_of_enrollment'},
                                            'training_hours':1
                                        }
                        },
                        # Stage 3 - Group by year
                        {
                            '$group':{
                                        '_id':'$Year',
                                        'avg_hours':{'$avg':'$training_hours'}
                                    }
                        },
                        # Stage 4 - Sort by avg_hours
                        {
                            '$sort':{'avg_hours':-1}
                        }
                    ]
                )

# Print result
for doc in result:
    pp.pprint(doc)

{'_id': 2019, 'avg_hours': 66.88719309887193}
{'_id': 2020, 'avg_hours': 65.56362425049967}
{'_id': 2018, 'avg_hours': 65.4376775007889}


In [7]:
# Question
result = db.hr.aggregate(
                    [
                        # Stage 1 - Filter documents
                        {
                            '$match':{'date_of_enrollment':{'$gte':datetime(2018,1,1)}}
                        },
                        # Stage 2 - Project month
                        {
                            '$project':{
                                            'Month':{'$month':'$date_of_enrollment'},
                                            'training_hours':1
                                        }
                        },
                        # Stage 3 - Group by year
                        {
                            '$group':{
                                        '_id':'$Month',
                                        'avg_hours':{'$avg':'$training_hours'}
                                    }
                        },
                        # Stage 4 - Sort by avg_hours
                        {
                            '$sort':{'avg_hours':-1}
                        }
                    ]
                )

# Print result
for doc in result:
    pp.pprint(doc)

{'_id': 10, 'avg_hours': 68.19245773732119}
{'_id': 8, 'avg_hours': 67.96581196581197}
{'_id': 3, 'avg_hours': 67.915611814346}
{'_id': 5, 'avg_hours': 67.01259445843829}
{'_id': 7, 'avg_hours': 66.36752136752136}
{'_id': 1, 'avg_hours': 66.16834170854271}
{'_id': 9, 'avg_hours': 66.11168831168831}
{'_id': 6, 'avg_hours': 66.01838755304102}
{'_id': 2, 'avg_hours': 65.08137931034483}
{'_id': 4, 'avg_hours': 64.83753148614609}
{'_id': 12, 'avg_hours': 62.754189944134076}
{'_id': 11, 'avg_hours': 62.73071895424837}


---
We couldn't have answered the query with a single group stage.

----

In [8]:
# Question
result = db.hr.aggregate(
                    [
                        # Stage 1 - Filter documents
                        {
                            '$match':{'date_of_enrollment':{'$gte':datetime(2018,1,1)}}
                        },
                        # Stage 2 - Project month
                        {
                            '$project':{
                                            'Year':{'$year':'$date_of_enrollment'},
                                            'Month':{'$month':'$date_of_enrollment'},
                                            'training_hours':1
                                        }
                        },
                        # Stage 3 - Group by year
                        {
                            '$group':{
                                        '_id':{
                                                'Year':'$Year',
                                                'Month':'$Month'
                                                },
                                        'avg_hours':{'$avg':'$training_hours'}
                                    }
                        },
                        # Stage 4 - Sort by avg_hours
                        {
                            '$sort':{'avg_hours':-1}
                        }
                    ]
                )

# Print result
for doc in result:
    pp.pprint(doc)

{'_id': {'Year': 2019, 'Month': 8}, 'avg_hours': 71.2392156862745}
{'_id': {'Year': 2018, 'Month': 3}, 'avg_hours': 71.19574468085106}
{'_id': {'Year': 2020, 'Month': 10}, 'avg_hours': 70.04}
{'_id': {'Year': 2020, 'Month': 4}, 'avg_hours': 69.92885375494072}
{'_id': {'Year': 2019, 'Month': 6}, 'avg_hours': 69.02479338842976}
{'_id': {'Year': 2018, 'Month': 7}, 'avg_hours': 68.85069444444444}
{'_id': {'Year': 2020, 'Month': 5}, 'avg_hours': 68.67588932806323}
{'_id': {'Year': 2020, 'Month': 8}, 'avg_hours': 68.17605633802818}
{'_id': {'Year': 2020, 'Month': 1}, 'avg_hours': 68.11320754716981}
{'_id': {'Year': 2019, 'Month': 7}, 'avg_hours': 67.75}
{'_id': {'Year': 2018, 'Month': 9}, 'avg_hours': 67.4332129963899}
{'_id': {'Year': 2019, 'Month': 1}, 'avg_hours': 67.29166666666667}
{'_id': {'Year': 2018, 'Month': 10}, 'avg_hours': 67.19433198380567}
{'_id': {'Year': 2019, 'Month': 10}, 'avg_hours': 67.1336032388664}
{'_id': {'Year': 2018, 'Month': 6}, 'avg_hours': 67.03056768558952}
{'_i

----
---
**To solve these questions we had to create two separate pipelines. This means we had to fetch the same documents twice from the database. Can we combine these two pipelines together and reduce the data we have to fetch from database?**

---
----

---

### Sub-pipelines using [$facet](https://docs.mongodb.com/manual/reference/operator/aggregation/facet/)

- `$facet` allows to create multiple aggregation sub-pipelines within a single stage.

- Each sub-pipeline can have multiple stages.

- Same documents are passed to all sub-pipelines from the preliminary stage to `$facet`.

- Output of one sub-pipeline within `$facet` is *not passed as input to the next sub-pipeline*.

- This way you would not have to fetch data from database for every aggregation.

- Useful for aggregating data along multiple dimensions.

----

**Syntax -**

{ $facet:

   {
   
      <outputField1>: [ <stage1>, <stage2>, ... ],
      
      <outputField2>: [ <stage1>, <stage2>, ... ],
      ...

   }
   
}

----

### Question -

Finding average training hours delivered per month and per year after 2017 using a single pipeline.


----

In [9]:
# Sub-pipeline

result = db.hr.aggregate(
    
    # Pipeline
    [
        # Stage 1 - Filter documents after 2017     
        {
            '$match':{'date_of_enrollment':{'$gte':datetime(2018,1,1)}}
        },
        
        # Stage 2 - Project year and month
        {
            '$project':{
                            'Year':{'$year':'$date_of_enrollment'},
                            'Month':{'$month':'$date_of_enrollment'},
                            'training_hours':1
                        }
        },
        
        # Stage 3 - Sub-pipelines
        {
            '$facet':{
                        # Sub-pipeline 1 - Aggregate along year
                        'Yearly_avg_hours':[
                                                # Stage 1 - Group
                                                {
                                                    '$group':{
                                                                '_id':'$Year',
                                                                'avg_hours':{'$avg':'$training_hours'}
                                                            }
                                                },
                                                # Stage 2 - Sort
                                                {
                                                    '$sort':{'avg_hours':-1}
                                                }
                                         ],
                
                        # Sub-pipeline 2 - Aggregate along month
                        'Monthly_avg_hours':[
                                                # Stage 1 - Group
                                                {
                                                    '$group':{
                                                                '_id':'$Month',
                                                                'avg_hours':{'$avg':'$training_hours'}
                                                            }
                                                },
                                                # Stage 2 - Sort
                                                {
                                                    '$sort':{'avg_hours':-1}
                                                }
                                         ] 
                        }
            }
    ])

# Print results
for doc in result:
    pp.pprint(doc)

{'Yearly_avg_hours': [{'_id': 2019, 'avg_hours': 66.88719309887193},
                      {'_id': 2020, 'avg_hours': 65.56362425049967},
                      {'_id': 2018, 'avg_hours': 65.4376775007889}],
 'Monthly_avg_hours': [{'_id': 10, 'avg_hours': 68.19245773732119},
                       {'_id': 8, 'avg_hours': 67.96581196581197},
                       {'_id': 3, 'avg_hours': 67.915611814346},
                       {'_id': 5, 'avg_hours': 67.01259445843829},
                       {'_id': 7, 'avg_hours': 66.36752136752136},
                       {'_id': 1, 'avg_hours': 66.16834170854271},
                       {'_id': 9, 'avg_hours': 66.11168831168831},
                       {'_id': 6, 'avg_hours': 66.01838755304102},
                       {'_id': 2, 'avg_hours': 65.08137931034483},
                       {'_id': 4, 'avg_hours': 64.83753148614609},
                       {'_id': 12, 'avg_hours': 62.754189944134076},
                       {'_id': 11, 'avg_hours': 62.7307

---
### Question - 

What is average quarterly and half-yearly training hours delivered for enrollees after 2017?

-----

In [10]:
# Question

result = db.hr.aggregate(
    
    # Pipeline
    [
        # Stage 1 - Filter documents    
        {
            '$match':{'date_of_enrollment':{'$gte':datetime(2018,1,1)}}
        },
        
        # Stage 2 - Prooject fields
        {
            '$project':{
                            'Month':{'$month':'$date_of_enrollment'},
                            'training_hours':1
                        }
        },
        
        # Stage 3 - Sub-pipelines
        {
            '$facet':{
                        # Sub-pipeline 1 - Half-yearly result 
                        'Half_Yearly_avg_hours':[
                                                # Stage 1
                                                {
                                                    '$bucket':{
                                                                    'groupBy': '$Month',
                                                                    'boundaries':[1, 7,13],
                                                                    'output':{
                                                                                'avg_hours':{'$avg':'$training_hours'}
                                                                            }
                                                              }
                                                },
                                                # Stage 2
                                                {
                                                    '$sort':{'avg_hours':-1}
                                                }
                                             ],
                
                        # Sub-pipeline 2 - Quarterly result
                        'Quarterly_avg_hours':[
                                                # Stage 1
                                                {
                                                    '$bucket':{
                                                                    'groupBy': '$Month',
                                                                    'boundaries':[1, 4, 7, 10, 13],
                                                                    'output':{
                                                                                'avg_hours':{'$avg':'$training_hours'}
                                                                            }
                                                              }
                                                },
                                                # Stage 2
                                                {
                                                    '$sort':{'avg_hours':-1}
                                                }
                                             ] 
                        }
            }
    ])

# Print results
for doc in result:
    pp.pprint(doc)

{'Half_Yearly_avg_hours': [{'_id': 1, 'avg_hours': 66.15992931301082},
                           {'_id': 7, 'avg_hours': 65.75483039931301}],
 'Quarterly_avg_hours': [{'_id': 7, 'avg_hours': 66.82931893687707},
                         {'_id': 1, 'avg_hours': 66.37186379928315},
                         {'_id': 4, 'avg_hours': 65.95381263616558},
                         {'_id': 10, 'avg_hours': 64.6048888888889}]}


----
### Exercise - 

Find average training hours delivered per education discipline and per education level for enrollees who joined from 2016 ownwards using sub-pipelines.

----